In [1]:
import pandas as pd
import numpy as np

Most of this is adapted directly from FTMSVisualization (https://github.com/wkew/FTMSVisualization) especially 0-FormulaGenerator.py and 1-FormulaAssignment.py files that are the ones relevant to what we want to achieve - make a "database" of possible formulas with their respective exact theoric mass and then assign to a set of m/z peaks a specific formula from said database.

Apart from this, the 2007 paper "Seven Golden Rules for heuristic filtering of molecular formulas obtained by accurate mass spectrometry" of Tobias Kind and Oliver Fiehn was used as a base for the rules used to make the "formula database". This was complemented with some criteria used by Kujawinski et al. (2006) for their CIA software for formula assignment.

### Warning!

This notebook takes a moderate amount of time to run with the current parameters and it asks a bit of RAM memory space. 

Update: Speed of the functions was considerably improved (still takes a good amount of time). RAM and space issues stil exist.


## Part nº 1 - Building the Formula "Database"

### Setting up functions to build the formula database

First a dictionary with all the masses and abundances of different elements (taken from FTMSVisualization).

In [2]:
#Atomic masses - https://ciaaw.org/atomic-masses.htm
#Isotopic abundances - https://ciaaw.org/isotopic-abundances.htm / https://www.degruyter.com/view/journals/pac/88/3/article-p293.xml
#isotoptic abundances from Pure Appl. Chem. 2016; 88(3): 293–306, Isotopic compositions of the elements 2013 (IUPAC Technical Report), doi: 10.1515/pac-2015-0503
#electron mass from NIST http://physics.nist.gov/cgi-bin/cuu/Value?meu|search_for=electron+mass
chemdict = {'H':(1.0078250322, 0.999844),
            'C':(12.000000000, 0.988922),
            'N':(14.003074004, 0.996337),
            'O':(15.994914619, 0.9976206),
            'Na':(22.98976928, 1.0),
            'P':(30.973761998, 1.0),
            'S':(31.972071174, 0.9504074),
            'Cl':(34.9688527, 0.757647),
            'F':(18.998403163, 1.0),
            'C13':(13.003354835, 0.011078)} 

Function to calculate formulas exact masses (Update: Fixed an error - mass of P was counted twice)

In [3]:
def getmass(c,h,o,n,s,p,cl,f):
    "Get the exact mass for any formula."
    massC = chemdict['C'][0] * c
    massH = chemdict['H'][0] * h
    massO = chemdict['O'][0] * o
    massN = chemdict['N'][0] * n
    massS = chemdict['S'][0] * s
    massP = chemdict['P'][0] * p
    massCl = chemdict['Cl'][0] * cl
    massF = chemdict['F'][0] * f 

    massTotal = massC + massH + massO + massN + massS + massP + massCl + massF

    return massTotal

Function to calculate the natural abundances of the formulas made of all the most common isotopes for each element.

In [4]:
def getabun(c,h,o,n,s,cl,c13=False):
    "The natural abundance of the formulas made of all the most common isotopes for each element."
    abunC, abunH, abunO, abunN, abunS, abunCl = 1,1,1,1,1,1
    
    if c13 == False:
        if c > 0:
            abunC = chemdict['C'][1] ** c
    else:
        if c > 0:
            abunC = (chemdict['C'][1] ** (c-1)) * chemdict['C13'][1] * c

    if h > 0:
        abunH = chemdict['H'][1] ** h

    if o > 0:
        abunO = chemdict['O'][1] ** o

    if n > 0:
        abunN = chemdict['N'][1] ** n

    if s > 0:
        abunS = chemdict['S'][1] ** s
        
    if cl > 0:
        abunCl = chemdict['Cl'][1] ** cl

    abunTotal = abunC * abunH * abunO * abunN * abunS * abunCl
    return abunTotal

### Maximum number of elements and ranges for element proportions according to the 7 golden rules paper

In [5]:
#Common range
com_range = {'H/C':(0.2,3.1),'N/C':(0,1.3),'O/C':(0,1.2),'P/C':(0,0.3),'S/C':(0,0.8)} #99.7% of all existing formulas
#Extended range
ext_range = {'H/C':(0.1,6),'N/C':(0,4),'O/C':(0,3),'P/C':(0,2),'S/C':(0,3)} #99.99% of all existing formulas

### Maximum number of elements and ranges for element proportions as per the chemical constraints put upon on the MetaboScape

Issues: Adding Cl and F element leads to exponentially greater amount of formulas and time of analysis.

#### These limits were the ones imposed in the following functions for rule nº1 and rule nº5 of the 7 golden rules.

In [6]:
#Rules applied to the file:
maxC,maxH,maxO,maxN,maxS,maxP = 39,72,20,20,5,9
maxF, maxCl = 8,5
ms_range = {'H/C':(0.2,3.1),'N/C':(0,1.3),'O/C':(0,1.2),'P/C':(0,0.3),'S/C':(0,0.8),'F/C':(0,1.5), 'Cl/C':(0,0.8)}

### Function to give every formula possible and their exact mass between a certain mass interval and according to other limitations such as the 7 golden rules paper or specifically imposed conditions (adapted from FTMSVisualization) - form_calc

Rules nº 1, 2 (partially), 4, 5 and 6 currently followed.

Rule nº 7 - TMS check not applicable.

Rule nº 3 - Isotope pattern checking is very lightly applied on the Formula Assignment step (just accounts for one C(13) peak and it doesn't affect the monoisotopic peak assignment).

#### form_calc function applies all these rules, rules nº2 and 6 however have specific functions.

form cal includes rule nº 1, 2, 4, 5 and 6 of the 7 Golden Rules and two criteria presented by Kujawinski et al (2006) that made the CIA algorithm (Compound Identification Algorithm) part of the Formularity software used as base to compare the formula assignments between the two.

Kujawinski criteria nº 1 was changed to account for P as they do in the Formularity software (h <= (2c + n + p + 2)) (addition of p).
 
Kujawinski criteria nº 2 was lightened since Formularity software broke this rule many times ((n + o) <= 2c) (instead of c).

In [7]:
def form_calc(low, high, elem_range = com_range):
    """Calculates all formulas possible (according to some stipulations) between a certain mass interval.
    
       low: scalar; lower limit of the molecular mass of the formula.
       high: scalar, upper limit of the molecular mass of the formula.
       elem_range: dictionary; dictionary where keys are string of the ratios of certain elements (examples: 'H/C', 'O/C') and
    their values are a tuple with the minimum and maximum ratios they can have in a certain formula (rule nº5).
    
       return: dictionary where keys are exact masses and values are tuples with the overall abundance of the monoisotopic 
    'mass', the number of atoms of each elements (in a specific order), bool value to if the formula follows the valency rules 
    when its elements are in their most common valency, the abundance and mass of the isotope with 1 C(13) atom and a defaulted
    False that is meant to see if the C(13) isotope mass or the monoisotopic mass is being used.
    """
    
    """Following the maximum elements from the recommended in 7 Golden Rules paper."""
    #RULE Nº 1 - Following the 7 golden rule limits
    if high <= 500:
        maxC,maxH,maxO,maxN,maxS,maxP = 39,72,20,20,10,9
    elif high <= 1000: #36 7 golden rules?
        maxC,maxH,maxO,maxN,maxS,maxP = 78,126,27,25,14,9
    #else: #elif m < 2000:
        #maxC,maxH,maxO,maxN,maxS,maxP = 156,180,63,32,14,9

    """Following the maximum elements from the limits imposed to MetaboScape."""
   # maxC,maxH,maxO,maxN,maxS,maxP = 39,72,20,20,5,9
    maxF, maxCl = 0,0
    
    maxC = min((int(high) / 12), maxC + 1) #max carbon nº is the smaller of the total mass/12 or predefined maxC
    maxH = min((maxC * 4), maxH + 1) #max hydrogen nº is the smaller of 4 times the nº of carbons or the predefined max hydrogen nº
    maxO2 = min((int(high) / 16), maxO + 1)#max oxygen nº has to be the smaller of the total mass/16 or predefined maxO
    #Those 3 above should have a +1 next to their maxC, maxH and maxO but since I forgot on the first 
    maxN2 = maxN + 1
    maxS2 = maxS + 1
    maxP2 = maxP + 1
    maxF2 = maxF + 1
    maxCl2 = maxCl + 1

    allposs = {} #pd.DataFrame(columns = ['abundance', 'c','h','o','n','s','p'])
    
    for c in range(int(maxC))[1:]: #molecules contain at least 1 C and 1 H
        for h in range(int(maxH))[1:]:
            hcrat = float(h)/float(c)
            if elem_range['H/C'][0] < hcrat < elem_range['H/C'][1]: #RULE Nº 4
                maxP = min(maxP2, int(c * elem_range['P/C'][1]+0.99)) #RULE Nº 5
                for p in range(maxP):
                    maxO = min(maxO2, int(c * elem_range['O/C'][1]+0.99)) #RULE Nº 5
                    for o in range(int(maxO)):
                        maxN = min(maxN2, int(c * elem_range['N/C'][1]+0.99)) #RULE Nº 5
                        for n in range(maxN):
                            maxS = min(maxS2, int(c * elem_range['S/C'][1]+0.99)) #RULE Nº 5
                            #Kujawinski Criteria nº 1 and 2
                            if h <= (2*c + n + p + 2):
                                if (n + o) <= 2*c: 
                                    for s in range(maxS):
                                        NOPS_ratio = NOPS(n,o,p,s)
                                        if NOPS_ratio:#RULE Nº 6 - element probability check - see function below
                                            maxF = min(maxF2, int(c * elem_range['F/C'][1]+0.99)) #RULE Nº 5
                                            for f in range(maxF):
                                                maxCl = min(maxCl2, int(c * elem_range['Cl/C'][1]+0.99)) #RULE Nº 5
                                                for cl in range(maxCl):
                                                    mass = getmass(c,h,o,n,s,p,cl,f) #getting the exact mass of the formula
                                                    if low < mass < high: #If the mass is in the given range
                                                        #formula = "C%iH%iO%iN%iS%iP%i" % (c,h,o,n,s,p)
                                                        Valency, Valency_normal = Lewis_Senior_rules(c,h,o,n,s,p,cl,f)
                                                        if Valency:
                                                            abundance = getabun(c,h,o,n,s,cl) #If isotope pattern checking becomes a thing
                                                            abundanceC13 = getabun(c,h,o,n,s,cl,c13=True)
                                                            massC13 = mass + chemdict['C13'][0] - chemdict['C'][0]
                                                            allposs[mass] = (abundance,c,h,o,n,s,p,cl,f,Valency_normal,abundanceC13,massC13,False)
        #print(c)
    return allposs

### Rule nº 6 - HNOPS heuristic probability check

FTMSVisualization didn't impose this rule. Instead, they did something else where they said that the total number of heteroatoms had to be smaller than 1.3 times the number of carbons. I altered to this to follow the 7 golden rules paper more closely. The following function sees if there are at least 3 types of heteroatoms in the formula and then observes if each of their counts is below a threshold shown in the 7 golden rules paper. If it isn't, NOPS_ratio is False and the formula is ignored.

#### Possible problem:

No extra heteroatom checks for F and Cl elements. This means that they are basically treated as non-heteroatoms so far. There should probably be soem kind of check that takes into account their presence when other heteroatoms are also present but I haven't found something like that in the literature yet.

In [8]:
#Rule nº 6 - HNOPS heuristic probability check
def NOPS (n,o,p,s):
    """Checks if the element counts follow the HNOPS heuristic probablility checks as delineated by the 7 golden rules paper.
    
       n,o,p,s - integers; number of N, O, P and S atoms respectively in the considered formula.
       
       returns: bool; True if it fulfills the conditions, False if it doesn't."""
    
    NOPS_ratio = True
    if (n > 1) and (o > 1) and (p > 1) and (s > 1): #NOPS
        if (n < 10) and (o < 20) and (p < 4) and (s < 3):
            NOPS_ratio = True
        else:
            NOPS_ratio = False
    elif (n > 3) and (o > 3) and (p > 3): #NOP
        if (n < 11) and (o < 22) and (p < 6):
            NOPS_ratio = True
        else:
            NOPS_ratio = False
    elif (o > 1) and (p > 1) and (s > 1): #OPS
        if (o < 14) and (p < 3) and (s < 3):
            NOPS_ratio = True
        else:
            NOPS_ratio = False
    elif (n > 1) and (p > 1) and (s > 1): #PSN
        if (n < 10) and (p < 4) and (s < 3):
            NOPS_ratio = True
        else:
            NOPS_ratio = False
    elif (n > 6) and (o > 6) and (s > 6): #PSN
        if (n < 19) and (o < 14) and (s < 8):
            NOPS_ratio = True
        else:
            NOPS_ratio = False

    return NOPS_ratio

### Rule nº 2 - Lewis and Senior (chemical rules) check

FTMSVisualization didn't impose this rule strictly . Instead, they checked the sum of the H and N (Na and K too, ionic compositions) elements to see if the formula followed a strict N rule, see if the sum was odd (not allowed) or even (allowed).

I altered this to follow the 7 golden rules paper more closely which says the following:

i) The sum of valences or the total number of atoms having odd valences is even;  - (close to the N rule followed)

ii) The sum of valences is greater than or equal to twice the maximum valence;

iii) The sum of valences is greater than or equal to twice the number of atoms minus 1 - (see if we can actually make a structure with those molecules - exaggerated since we are considering all molecules can always have their maximum valence state.


While "allowing maximum valence states for each element" i.e. allowing for elements to be in higher valences that they can assume in some molecules and not only their ground states (example: N usually has a valence of 3 establishing 3 bonds, but can have a valence of 4 (molecules with NO2 groups) or 5). Maximum valences for each element is an absolute value.

Rule ii) As I've seen in a paper ('ANALOGOUS ODD-EVEN PARITIES IN MATHEMATICS AND CHEMISTRY', 2003, Morikawa T and Newbold BT), the objective of this rule is to indicate "the non-existence of small molecules such as CH2" (although they don't expalin what maximum valence is). Since we are dealing with molecules with, at least, 100 Da, I ignored this rule so far.

The function so makes a minimum requirement for the formula to fulfill all these rules with their elements in the maximum valency and gives an indication if the formula also follows these rules with their elements in their most common valency.

In [9]:
def Lewis_Senior_rules(c,h,o=0,n=0,s=0,p=0,cl=0,f=0):
    """See if the formula follows Lewis' and Senior's rules (considering all max possible valency states for each element
    except Cl).
    
       c,h,o,n,s,p,cl,f - integers; number of C, H, O, N, S, P, Cl and F atoms respectively in the considered formula.
       
       returns: (bool, bool); (considering max valency in each element, considering normal valency of each element), True if
    it fulfills the conditions, False if it doesn't."""
    
    #Max_Valencies
    valC = 4
    valH = 1 #ODD
    valO = 2 #Positive instead of negative for it to work?
    valN, max_valN = 3, 5 #Normally 3, ODD
    valS, max_valS = 2, 6 
    valP, max_valP = 3, 5 #Normally -3, Odd
    valCl, max_valCl = 1, 7 #7 #Normally -1, Odd ????
    valF = 1 #Normally -1, Odd
    
    Valency = False
    Valency_normal = False
    #1st rule - The sum of valences or the total number of atoms having odd valences is even.
    if (h + n + p + cl + f) % 2 == 0:
        #Elements with their max valences
        total_max_v = (valC * c) + (valH * h) + (valO * o) + (max_valN * n) + (max_valS * s) + (max_valP * p) + (max_valCl * cl) + (valF * f)
            #2nd rule - The sum of valences is greater than or equal to twice the maximum valence.
            #Ok, not applying this one since it normally only eliminates small molecules either way and we are searching for molecules
            #with more than 100 Da.

            #3rd rule - The sum of valences is greater than or equal to twice the number of atoms minus 1.
        natoms = c + h + o + n + s + p + cl + f
        if total_max_v >= (2*(natoms-1)):
            Valency = True
        
        if Valency:
            #Elements with their common valences
            total_v = (valC * c) + (valH * h) + (valO * o) + (valN * n) + (valS * s) + (valP * p) + (valCl * cl) + (valF * f)
            #3rd rule - The sum of valences is greater than or equal to twice the number of atoms minus 1.
            #natoms = c + h + o + n + s + p + cl + f
            if total_v >= (2*(natoms-1)):
                Valency_normal = True

    return Valency, Valency_normal

Making a dictionary of dataframes with each dataframe having all possible formulas (according to the previously mentioned rules and functions) of a 100 mass interval with the name of each dataframe being dict + lower (normal) limit of said mass interval.

#### Warning: Don't do range (1,12) here

In [10]:
formulas = {}
dif = chemdict['C13'][0] - chemdict['C'][0]
#Change range here - don't do range(1,12)!!
#Example range(1,4), gives in the formula dictionary 3 DataFrames with formulas with m/z between 100-200, 200-300 and 300-400
for i in range(1,8):
    #i*100 - dif - 0.001 = the idea here is to account for possible C(13) isotopes that might surpass the i*100 barrier while
    #the monoisotopic formula wouldn't which would lead to an error in the formula assignment procedure if the monoisotopic
    #isn't present in the database. That does mean there is a slight overlap between the different files.
    a = form_calc(i*100 - dif - 0.001, (i+1)*100, elem_range = ms_range)
    formulas['dict' + str(i*100)] = pd.DataFrame.from_dict(a, orient = 'index', columns = ['Abundance', 'C','H','O','N','S','P',
                                                                                           'Cl','F','Valency','C13 Abun','C13 mass','C13 check'])
    formulas['dict' + str(i*100)] = formulas['dict' + str(i*100)].sort_index()
    print(i*100, 'complete')
    #Writing a .csv file of the data
    #formulas['dict' + str(i*100)].to_csv('dict' + str(i*100) + '.csv')


100 complete
200 complete
300 complete
400 complete
500 complete
600 complete
700 complete


Writing individual files if one wants to. 

In [11]:
formulas

{'dict100':             Abundance  C   H  O  N  S  P  Cl  F  Valency  C13 Abun  \
 99.014270    0.904952  4   5  0  1  1  0   0  0     True  0.040549   
 99.023786    0.952024  4   6  0  1  0  1   0  0     True  0.042659   
 99.032028    0.947647  4   5  2  1  0  0   0  0     True  0.042463   
 99.043262    0.953524  3   5  1  3  0  0   0  0     True  0.032044   
 99.068414    0.938798  5   9  1  1  0  0   0  0     True  0.052583   
 ...               ... ..  .. .. .. .. ..  .. ..      ...       ...   
 199.995702   0.909120  7   4  7  0  0  0   0  0     True  0.071288   
 199.996572   0.821305  8   8  2  0  2  0   0  0     True  0.073603   
 199.997845   0.868018  8  10  0  0  1  2   0  0     True  0.077789   
 199.998694   0.918983  6   5  4  2  0  1   0  0     True  0.061767   
 199.999943   0.806598  5  12  2  0  3  0   0  0     True  0.045178   
 
               C13 mass  C13 check  
 99.014270   100.017625      False  
 99.023786   100.027141      False  
 99.032028   100.035383 

## Part nº2 - Assigning Formulas to m/z peaks / bucket labels

Reading result files from Formularity. Two main datasets were used - first the test set that comes with formularity (test_peaks) and the second file is the first replicate of the CAN grapevine variety spectrum obtained in Negative mode.

In [12]:
#Original file in the example_results of Formularity as an example
formula_file = pd.read_csv('report.csv')

electron_mass = 0.0005485799#09065

formula_file[formula_file['El_comp'].notnull()]

,Mass,C,H,O,N,C13,S,P,Na,El_comp,Class,NeutralMass,Error_ppm,Candidates,test_peaks
132,157.050644,7,10,4,0,0,0,0,0,CHO,Lignin,158.057920,-0.071325,1,12366396.0
154,165.019322,8,6,4,0,0,0,0,0,CHO,ConHC,166.026599,0.060814,1,6988142.5
155,166.022685,7,6,4,0,1,0,0,0,CHO,ConHC,167.029962,0.011929,0,1288458.1
162,169.123406,10,18,2,0,0,0,0,0,CHO,Lipid,170.130682,-0.014880,1,10983488.0
170,171.139061,10,20,2,0,0,0,0,0,CHO,Lipid,172.146337,-0.042756,1,1224656.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8365,989.438943,50,70,20,0,0,0,0,0,CHO,Lignin,990.446219,-0.175998,-1,2409162.0
8446,1007.750553,61,106,5,2,0,0,2,0,CHONP,Lipid,1008.757830,-0.131214,-1,2296548.0
8538,1038.688555,60,97,13,1,0,0,0,0,CHON,Lipid,1039.695832,0.154205,-1,2610562.3
8633,1130.991164,70,133,9,1,0,0,0,0,CHON,Lipid,1131.998440,-0.358170,-1,2300583.5


In [13]:
#Test peak files with parameters: S+P Formula score, 0,5 ppm, 700 limit database
Test_SP_05 = pd.read_csv('testpeaks_report.csv')
Test_SP_05[Test_SP_05['El_comp'].notnull()]

#Test peak files with parameters: N+S+P Formula score, 1 ppm, 800 limit database
Test_NSP_10 = pd.read_csv('TESTP_NSP_1_800.csv')
Test_NSP_10[Test_NSP_10['El_comp'].notnull()]

#CAN file with parameters: S+P Formula score, 1 ppm, 800 limit database
SP_10 = pd.read_csv('CAN_SP_1_800.csv')
SP_10[SP_10['El_comp'].notnull()]

#CAN file with parameters: S+P Formula score, 0,5 ppm, 800 limit database
SP_05 = pd.read_csv('CAN_SP_05.csv')
SP_05[SP_05['El_comp'].notnull()]

#CAN file with parameters: N+S+P Formula score, 0,5 ppm, 800 limit database
NSP_05 = pd.read_csv('CAN_NSP_0.5_800.csv')
NSP_05[NSP_05['El_comp'].notnull()]

#CAN file with parameters: N+S+P Formula score, 1 ppm, 800 limit database
NSP_10 = pd.read_csv('CAN_NSP_1_800.csv')
NSP_10[NSP_10['El_comp'].notnull()]

,Mass,C,H,O,N,C13,S,P,Na,El_comp,Class,NeutralMass,Error_ppm,Candidates,CAN1 (1)
48,171.10276,9,16,3,0,0,0,0,0,CHO,Protein,172.110036,-0.535047,1,542141
61,185.09613,7,15,0,4,0,0,1,0,CHNP,Lipid,186.103406,0.142638,1,2759693
63,187.09777,9,16,4,0,0,0,0,0,CHO,Protein,188.105046,-0.996597,1,883076
68,238.03067,7,14,4,1,0,1,1,0,CHONSP,AminoSugar,239.037946,0.705646,2,1601493
73,239.03565,7,8,2,6,0,1,0,0,CHONS,Lignin,240.042926,0.073967,1,559354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,747.49423,57,64,0,0,0,0,0,0,CH,UnsatHC,748.501506,-0.941097,15,882589
563,755.28188,41,44,12,2,0,0,0,0,CHON,Lignin,756.289156,0.354919,38,708393
564,761.44441,37,66,14,2,0,0,0,0,CHON,Protein,762.451686,-0.369372,18,807533
565,771.25104,38,44,17,0,0,0,0,0,CHO,Lignin,772.258316,-0.604045,50,1014089


### Function to assign a formula - form_checker 

How the function works:

1) Select formulas that are within the threshold (in ppm - thresppm) chosen.

2) See if there is a possibility that the peak might be an isotope of a previously assigned formula.

3) If no, go to step 7)

4) If yes and the intensity of the peak is very close to the expected based on the relative natural abundancies from the isotopes between the monoisotopic peak and the C(13) peak, the formula is right away assigned to the peak.

5) If yes and the intensity of the peak is somewhat close to the expected based on the relative natural abundancies from the isotopes between the monoisotopic peak and the C(13) peak, the formula is added to the list of possible formulas got in 1) to pass the different formula criteria in step 7).

6) If yes but the intensity of the peak is very far from the expected, the formula isn't added to the list of possible formulas got in 1).

7) Different criteria for formula assignment are made and the best candidate from the list of possible formulas is assigned to the peak. The criteria and checks used are the following (in order):

All criteria used are skipped if no formula present in the candidates pass a specific criterion. If only one formula passes a specific criterion, that formula is assigned and all further criteria are skipped.

- Check nº 1: Just a check for if there are multiple candidates, if there is only one candidate, that formula is assigned, if there isn't any, no formula is assigned.

- Check nº 2: Check if the candidate formulas follow valency rules with their elements in their most common valency.

- Check nº 3: Candidate formulas with less number of different heteroatoms are prioritized - first formulas with only C, H, O and N and then formulas with only C, H, O, N, S and P. This check was taken out since it worsens comparisons.

- Check nº 4: Prioritize formulas with lowest absolute number of heteroatoms - counting N, S, P (Cl and F) to mimick Formularity.

- Check nº 5: Choose from the remaining formulas, the one with the lowest absolute error between theoretical and experimental mass.

In [14]:
def form_checker(mass, threshppm, df, intensity, assignments, isotope_check=True):
    """Assigning formulas to an m/z peak based on the distance of the m/z to the formulas present in a given database.
    
       mass: scalar; m/z of the peak.
       threshppm: scalar; error threshold for formulae in ppm - i.e. relative error threshold.
       df: Pandas Dataframe; dataframe with the formulas that are possible to assign to the m/z peak.
       intensity: scalar; intensity of the peak.
       assignments: Pandas Dataframe; previous formula assignments made in the dataset.
       isotope_check: bool (default: True); 
       
       returns: tuple with the mass given, the formula assigned (np.nan if no formula could be assigned within the 2 given
    thresholds) and the intensity of the peak."""
    
    #Calculate mass difference allowed based on the mass and ppm thresholds given
    mass_dif = mass - (mass*1000000/(1000000 + threshppm))
    #Select the formulas from the database dataframe that are within said mass difference to the mass given
    df2 = df.copy()
    df2 = df2[df2.index<= (mass+mass_dif)]
    df2 = df2[df2.index>= (mass-mass_dif)]
    #print(df2)
    
    #Isotope Checker
    if isotope_check == True:
        #See if there are previously assigned formulas that could have a C(13) isotope around the mass 
        
        #This is actually the mass of a monoisotopic peak
        monoiso_mass = mass - chemdict['C13'][0] + chemdict['C'][0]
        monoiso_mass_dif = monoiso_mass - (monoiso_mass*1000000/(1000000 + threshppm)) #Some doubts here
        #isos renamed to iso_candidates and close_iso renamed to close_iso_candidates for more clarity
        #Possible previously assigned formulas that are candidate to have the C(13) isotope around the mass
        iso_candidates = forma[forma.index<= (monoiso_mass + mass_dif + monoiso_mass_dif)]
        iso_candidates = iso_candidates[iso_candidates.index >= (monoiso_mass - mass_dif - monoiso_mass_dif)] 
        #Only consider peaks that had formulas assigned previously
        iso_candidates = iso_candidates[iso_candidates['Form_give'].notnull()]
        #print(isos)
        
        if len(iso_candidates) != 0: #If there were possible candidates
            #Not consider previously assigned peaks that already were C(13) isotopes of other peaks
            iso_candidates = iso_candidates[~iso_candidates['Form_give'].str.endswith('C(13)')]
            #print(isos, intensity)
            close_iso_candidates = pd.DataFrame(columns = df2.columns)
            
            for i in range(len(iso_candidates)):
                #theo_relative_intensity = the theoretical relative intensity between C(13) isotope and monoisotopic peaks 
                #according to natural abundancy.
                theo_relative_intensity = df.loc[iso_candidates['Theo_mass'].iloc[i]].iloc[-3] / df.loc[
                    iso_candidates['Theo_mass'].iloc[i]].iloc[0]
                #exp_relative_intensity = the experimental relative intensity between the two peaks
                exp_relative_intensity = intensity / iso_candidates['Intensity'].iloc[i]
                #print(exp_relative_intensity, theo_relative_intensity)
                
                #See if the experimental relative intensity is close to the theoretical.
                if (exp_relative_intensity > (theo_relative_intensity - 0.6*theo_relative_intensity) 
                    and exp_relative_intensity < (theo_relative_intensity + 0.6)):
                    #print(df.loc[isos['Theo_mass'].iloc[i]])
                    
                    #Add to the candidate formulas the C(13) isotope information by modifying the monoisotopic mass information
                    df2.loc[df.loc[iso_candidates['Theo_mass'].iloc[i]].iloc[-2]] = df.loc[iso_candidates['Theo_mass'].iloc[i]]
                    df2.loc[df.loc[iso_candidates['Theo_mass'].iloc[i]].iloc[-2],'C13 check'] = 1
                    df2.loc[df.loc[iso_candidates['Theo_mass'].iloc[i]].iloc[-2],'C'] = df.loc[
                        iso_candidates['Theo_mass'].iloc[i]].loc['C'] - 1
                    
                    #See if the experimental relative intensity is very close to the theoretical.
                    if (exp_relative_intensity > (theo_relative_intensity - 0.2*theo_relative_intensity) and 
                        exp_relative_intensity < (theo_relative_intensity + 0.2*theo_relative_intensity)):
                        close_iso_candidates.loc[df.loc[iso_candidates['Theo_mass'].iloc[i]].iloc[-2]] = df2.loc[df.loc[iso_candidates['Theo_mass'].iloc[i]].iloc[-2]]
                        #print(close_isos)
            
            #If there are very close isotopes candidates, the formulas assigned will be one of them
            if len(close_iso_candidates) != 0:
                if len(close_iso_candidates) == 1:
                    df2 = close_iso_candidates
                    return(mass, formulator(df2['C'].values[0],df2['H'].values[0],df2['O'].values[0],df2['N'].values[0],
                                            df2['S'].values[0],df2['P'].values[0],df2['F'].values[0],df2['Cl'].values[0],
                                            df2['C13 check'].values[0]),
                          df2.index[0])#,
                else:
                    df2 = close_iso_candidates
    
    #Criteria for more than one possiblity: Check number of possible formulas, normal_valency allows formula, check formulas
    #with low amounts of different heteroatoms, check formulas with lowest amounts of heteroatoms, make smallest error check.
    
    #Check nº 1: If more than one formula exists:
    if len(df2) > 1:
        #Check nº 2: if there are formulas that can exist with elements in their most common valency
        #Con: Formulas with groups NO2 where N has a valency of 4 will be excluded
        df_valency = df2[df2['Valency'] == True]
        if len(df_valency) != 0:
            df2 = df_valency
            if len(df2) == 1:
                return(mass, formulator(df2['C'].values[0],df2['H'].values[0],df2['O'].values[0],df2['N'].values[0],
                                        df2['S'].values[0],df2['P'].values[0],df2['F'].values[0],df2['Cl'].values[0],
                                        df2['C13 check'].values[0]),
                      df2.index[0])#,
                                        #df2['C13'].values[0]))

#        #Check if the formulas follows the stricter Kujawinski criterion - doesn't help  
#        df_no = df2[(df2['N'] + df2['O']) <= df2['C']]
#        if len(df_no) != 0:
#            df2 = df_no
#            if len(df2) == 1:
#                return(mass, formulator(df2['C'].values[0],df2['H'].values[0],df2['O'].values[0],df2['N'].values[0],
#                                        df2['S'].values[0],df2['P'].values[0],df2['F'].values[0],df2['Cl'].values[0],
#                                        df2['C13 check'].values[0]),
#                      df2.index[0])#,
        
#        #Check nº 3: if there are formulas with low amounts of different heteroatoms: first prioritize C,H,O,N. - doesn't help
#        #df_CHON = df2[df2['S'] + df2['P'] + df2['F'] + df2['Cl'] == 0]
#        df_CHON = df2[df2['S'] + df2['P'] + df2['F'] + df2['Cl'] + df2['N'] == 0]
#        if len(df_CHON) != 0:
#            #df2 = df_CHON
#            if len(df2) == 1:
#                return(mass, formulator(df2['C'].values[0],df2['H'].values[0],df2['O'].values[0],df2['N'].values[0],
#                                        df2['S'].values[0],df2['P'].values[0],df2['F'].values[0],df2['Cl'].values[0],
#                                        df2['C13 check'].values[0]),
#                      df2.index[0])#,
                                        #df2['C13'].values[0]))
            
#        #Check nº 3 part 2: if there are formulas with low amounts of different heteroatoms: prioritize C,H,O,N,S,P.
#        #Not applicable
#        df_CHONSP = df2[df2['F'] + df2['Cl'] == 0]
#        df4 = df4[df4['Cl'] == 0]
#        if len(df_CHONSP) != 0:
#            df2 = df_CHONSP
#            if len(df2) == 1:
#                return(mass, formulator(df2['C'].values[0],df2['H'].values[0],df2['O'].values[0],df2['N'].values[0],
#                                        df2['S'].values[0],df2['P'].values[0],df2['F'].values[0],df2['Cl'].values[0]))
        
        if len(df2) > 1:
            #Check nº 4: Lowest Heteroatom count
            df2 = df2[df2[['N','S','P','F','Cl']].sum(axis = 1) == 
                      df2[['N','S','P','F','Cl']].sum(axis = 1).min()]
            #df2 = df2[df2[['S','P']].sum(axis = 1) == 
            #          df2[['S','P']].sum(axis = 1).min()]
            if len(df2) == 1:
                return(mass, formulator(df2['C'].values[0],df2['H'].values[0],df2['O'].values[0],df2['N'].values[0],
                                        df2['S'].values[0],df2['P'].values[0],df2['F'].values[0],df2['Cl'].values[0],
                                        df2['C13 check'].values[0]),
                      df2.index[0])#,
                                        #df2['C13 check'].values[0]))
            
            #Final Check: Lowest error
            #Calculate and store the error (in ppm) of the mass of the filtered formulas
            df3 = pd.DataFrame(abs(((mass - df2.index)/df2.index)*1000000), index = df2.index, columns = ['error'])
            mini = df3.idxmin(axis = 'index')
            #Choose the formula with the lowest error (closest to original mass).
            return(mass, formulator(df2.loc[mini, 'C'].values[0],df2.loc[mini,'H'].values[0],df2.loc[mini,'O'].values[0],
                                    df2.loc[mini,'N'].values[0],df2.loc[mini,'S'].values[0],df2.loc[mini,'P'].values[0],
                                    df2.loc[mini,'F'].values[0], df2.loc[mini,'Cl'].values[0],
                                    df2.loc[mini,'C13 check'].values[0]),
                   mini[0])
    
    #Only one formula is within the mass interval - it will be assigned
    elif len(df2) == 1:
        return(mass, formulator(df2['C'].values[0],df2['H'].values[0],df2['O'].values[0],df2['N'].values[0],df2['S'].values[0],
                                df2['P'].values[0],df2['F'].values[0],df2['Cl'].values[0], df2['C13 check'].values[0]),
              df2.index[0])
    
    #No formula is within the mass interval - no formula assigned
    else: 
        return(mass, np.nan, np.nan)

#### Formulator function

Function to write formulas based on the element counts in the same order that MetaboScape writes its formulas.

In [15]:
#Adapted from FTMSVisualization
def formulator(c,h,o,n,s,p,f=0,cl=0,c13=0):
    """Transforms element counts to a readable formula in string format. Element order: C, H, N, O, S, P, F, Cl (and C(13))."""
    
    formula = "C"+str(c)+"H"+str(h)

    if cl > 0:
        if cl > 1:
            formula = formula + "Cl" + str(cl)
        else:
            formula = formula + "Cl"
    if f > 0:
        if f > 1:
            formula = formula + "F" + str(f)
        else:
            formula = formula + "F"
    if n > 0:
        if n > 1:
            formula = formula + "N" + str(n)
        else:
            formula = formula + "N"
    if o > 0:
        if o > 1:
            formula = formula + "O" + str(o)
        else:
            formula = formula + "O"
    if p > 0:
        if p > 1:
            formula = formula + "P" + str(p)
        else:
            formula = formula + "P"
    if s > 0:
        if s > 1:
            formula = formula + "S" + str(s)
        else:
            formula = formula + "S"
    if c13 > 0:
        if c13 > 1:
            formula = formula + "C(13)" + str(s)
        else:
            formula = formula + "C(13)"
            
    return formula

### Parameters for formula attribution (FTMSVisualization)

In [16]:
#Define some parameters for our assignment thresholds.
#Relevant parameters
threshppm = 1.0 # Error threshold for formulae in ppm - i.e. relative error threshold.
#Not needed for new form_checker
threshold = 0.005 # Error threshold for formulae in Da - i.e. absolute error threshold.

#Parameters concerning Kendrick Mass Defect series that they used in a myriad of functions. I don't understand its relevancy.
maxgap = 0.0003 #gap between KMDs for separating series. Units are Da, hence the small numbers. This value will be a compromise, and depend on complexity of your dataset.
# Looking at Z* and KMD method compared to other methods it seems an appropriate range could be 0.0002 or even 0.00002. Look at closer in a future version
minKMDseries = 1 #minimum number of peaks in a single homologous series to assign by zstar approach # 3 seems a good number.

### Test with a very small part of the dataset

In [17]:
#teste = formula_file[formula_file['NeutralMass']< 200]
#teste

In [18]:
#teste_form = []
#for i in range(len(teste)):
    #mass = teste.iloc[i,0]
    #print(form_checker(mass, threshppm, df = formulas['dict100'])) #1st Form Checker function
    #print(form_checker(mass, threshppm, df = formulas['dict100'])) #2nd Form Checker function
    #teste_form.append(form_checker(mass, threshppm, df = formulas['dict100']))

In [19]:
#intersection = 0
#cls = 0
#for i in range(len(teste)):
#    if teste.iloc[i,3] == teste_form[i][1]:
#        intersection = intersection + 1
#intersection

### Assigning formula to the full dataset

Cut the intensity and samples out of the dataset

In [20]:
#Taking the necessary columns from files originating from the test_peaks file.
formula_file2 = Test_NSP_10[['NeutralMass','C','H','O','N','S','P','Na','C13','test_peaks']]

#Taking the necessary columns from files originating from the CAN file.
#formula_file2 = NSP_10[['NeutralMass','C','H','O','N','S','P','Na','C13','CAN1 (1)']]

In [21]:
#Make a column to store the formulas with each of the labels.
forma2 = pd.DataFrame(np.zeros((len(formula_file2),1)), index = formula_file2['NeutralMass'], columns = ['Formula'])

### Try to assign a formula to each peak on the dataset

#### Warning: Don't do range (1,12) here, do the same range as done in form_calc

In [22]:
#Change range here - don't do range(1,12)!!
#Same range as in the form_calc before
threshppm = 1
forma = pd.DataFrame(columns = ['Form_give','Intensity','Theo_mass'])
for i in range (1,8):
    teste = formula_file2[formula_file2['NeutralMass'] < (i+1)*100]
    teste = teste[teste['NeutralMass'] > i*100]
    for j in range(len(teste)):
        mass = teste.iloc[j,0]
        tup = form_checker(mass, threshppm, formulas['dict' + str(i*100)], teste.iloc[j,-1], assignments = forma) 
        forma.loc[tup[0]] = tup[1],teste.iloc[j,-1],tup[2]
        #print(j)
    print(i*100, 'complete')


100 complete
200 complete
300 complete
400 complete
500 complete
600 complete
700 complete


### Just until m/z 800 for this example

### Make a dataframe with Formulas assigned by Formularity ('Formula') and by form_checker ('Form_give')

In [23]:
formula_file3 = formula_file2.set_index('NeutralMass')
ending = pd.concat([formula_file3,forma['Form_give']], axis = 1)

In [24]:
#Transforming the assigned formulas from Formularity to the same format as the formulas assigned in the notebook.

forma2 = pd.DataFrame(np.zeros((len(formula_file2),1)), index = formula_file2['NeutralMass'], columns = ['Formula'])
#for i in len()
for i in range(len(ending)):
    if ending['C'].iloc[i] != 0:
        if ending['C13'].iloc[i] != 0:
            forma2.iloc[i] = formulator(ending['C'].iloc[i],ending['H'].iloc[i],ending['O'].iloc[i],
                                        ending['N'].iloc[i],ending['S'].iloc[i],ending['P'].iloc[i],c13 = ending['C13'].iloc[i])
        else:
            forma2.iloc[i] = formulator(ending['C'].iloc[i],ending['H'].iloc[i],ending['O'].iloc[i],
                                        ending['N'].iloc[i],ending['S'].iloc[i],ending['P'].iloc[i],c13 = ending['C13'].iloc[i])   
    else:
        forma2.iloc[i] = np.nan
#ending.loc[ending.index>398]
forma2.notnull()
ending

,C,H,O,N,S,P,Na,C13,test_peaks,Form_give
113.460906,0,0,0,0,0,0,0,0,5620136.5,NaN
113.820949,0,0,0,0,0,0,0,0,41591812.0,NaN
118.753600,0,0,0,0,0,0,0,0,1267222.9,NaN
118.936991,0,0,0,0,0,0,0,0,2812275.0,NaN
119.369588,0,0,0,0,0,0,0,0,19251212.0,NaN
...,...,...,...,...,...,...,...,...,...,...
1168.406517,0,0,0,0,0,0,0,0,2748621.8,NaN
1171.351234,0,0,0,0,0,0,0,0,2627437.5,NaN
1178.620294,0,0,0,0,0,0,0,0,2679755.3,NaN
1186.604555,0,0,0,0,0,0,0,0,2609406.8,NaN


Checking the results in diverse ways

In [25]:
ending = pd.concat([forma2,forma], axis = 1)
ending.replace({0:np.nan})
ending[ending['Formula'] != 0]
ending = ending.loc[ending.index<800]
ending = ending.loc[ending.index>100]
#ending

In [26]:
ending = ending.loc[ending.index<800]
ending = ending.loc[ending.index>100]
#ending = ending['C(13)' in ending['Formula']]
colus = ending[['Formula','Form_give']]

colus2 = colus.notnull()
print('-----------Number of formulas given by the Formularity (Formula) and this notebook (Form_give)---------------')
print(colus2.sum())
print('Total number of peaks:', len(ending))

-----------Number of formulas given by the Formularity (Formula) and this notebook (Form_give)---------------
Formula      6972
Form_give    6980
dtype: int64
Total number of peaks: 7341


### Number of equal formula assignments between Formularity and form_checker

In [27]:
intersection = 0
cls = 0
for i in range(len(colus)):
    if colus.iloc[i,0] == colus.iloc[i,1]:
        intersection = intersection + 1

print(intersection, 'em', colus2.sum().iloc[0])
print(intersection/colus2.sum().iloc[0]*100, '% of same attributions.', )

6477 em 6972
92.90017211703959 % of same attributions.


In [28]:
ddddd = colus[colus.fillna(0)['Formula'] != 0]
#ddddd = colus[colus.fillna(0)['Form_give'] != 0]
ccccc = ddddd[ddddd['Formula'] != ddddd['Form_give']]
oi = []
#oi = pd.DataFrame(columns = ['Formula'])
for i in range(len(ccccc)):
    if 'C(13)' in ccccc['Formula'].iloc[i]:
        oi.append(ccccc.iloc[i])
    #if 'C(13)' in ccccc['Form_give'].iloc[i]:
        #oi.append(ccccc.iloc[i])

len(oi)
len(oi) / (colus2.sum().iloc[0] - intersection)

0.701010101010101

In [29]:
ccccc.iloc[50:100]

,Formula,Form_give
385.137363,C17H25O7PC(13),C17H23NO9
385.173806,C18H29O6PC(13),C18H27NO8
387.106690,C16H22O8SC(13),C18H17N3O7
387.116681,C16H23O8PC(13),C16H21NO10
387.153050,C17H27O7PC(13),C17H25NO9
387.189391,C18H31O6PC(13),C18H29NO8
388.146655,C18H21N3O6C(13),C28H20O2
389.122435,C16H24O8SC(13),C18H19N3O7
389.150281,C19H24N3O4P,C27H20O2C(13)
389.168653,C17H29O7PC(13),C17H27NO9
